In [15]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

data_path = Path("../data/")   # full path to the HEF folder

train_df = pd.read_csv(data_path / "mimic_train_HEF.csv", low_memory=False)
test_df  = pd.read_csv(data_path / "mimic_test_HEF.csv",  low_memory=False)

train_df.shape, test_df.shape

((20885, 44), (5221, 39))

drop all unnecesarry colums (maybe later keep DOB for age)
SPo2 ook nog is nazien of we die er wel best uitlaten 

In [20]:
# --- 1. Columns to drop (from our earlier analysis)
drop_cols = ['LOS', 'subject_id', 'hadm_id', 'icustay_id', 'ADMITTIME','DISCHTIME', 'DEATHTIME', 'DOD', 'DOB', 'Diff', 'SpO2_Max']

# --- 2. Target column
target_col = 'HOSPITAL_EXPIRE_FLAG'

# --- 3. Create X (features) and y (target)
X = train_df.drop(columns=drop_cols + [target_col])
y = train_df[target_col]

# --- 4. Also drop the same columns from the test set
X_test = test_df.drop(columns=drop_cols, errors='ignore')

# --- 5. Confirm same feature columns in train/test
print("Train shape:", X.shape)
print("Test shape:", X_test.shape)
print("Same columns in train/test:", set(X.columns) == set(X_test.columns))


Train shape: (20885, 32)
Test shape: (5221, 32)
Same columns in train/test: True


In [21]:
import numpy as np

numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()

print("Numeric columns:", len(numeric_cols))
print("Categorical columns:", len(categorical_cols))


Numeric columns: 23
Categorical columns: 9


## Imputing missing data

In [22]:
X[numeric_cols] = X[numeric_cols].fillna(X[numeric_cols].median())
print("Remaining missing values:", X.isna().sum().sum())
# remaining is for the categorical colum 

Remaining missing values: 722


In [23]:
for col in categorical_cols:
    X[col] = X[col].fillna(X[col].mode()[0])
print("Remaining missing values after imputing categorical columns:", X.isna().sum().sum())

Remaining missing values after imputing categorical columns: 0


## outliers